## Импорт библиотек

In [1]:
# !pip install --upgrade sqlmagic
# !pip install --upgrade sqlalchemy
# !pip install --upgrade ipython-sql
# %load_ext sql

In [2]:
import sqlite3
from prettytable import PrettyTable
# import sqlalchemy

# %load_ext sql # загрузка расширения sql
# %reload_ext sql # отключение расширения sql

## Загрузка и подключение к базе данных

In [3]:
# скачивание базы данных с диска
# import gdown
# !gdown 'https://drive.google.com/uc?id=1pmfc0FrkMvlM2SITpFz6bilk-dvDG_y0'

In [4]:
# при помощи расширения sql
# %sql sqlite:/data/data.db

# %config SqlMagic.displaylimit = 50 # установка лимита вывода строк запроса

# при помощи connection
connection = sqlite3.connect('data/data.db')
cursor = connection.cursor()

Вывод списка имеющихся в базе таблиц

In [5]:
# %sql select tbl_name, "sql" from sqlite_master; # при помощи расширения sql

# при помощи connection
cursor.execute('''
SELECT 
  tbl_name,
  "sql" 
FROM 
  sqlite_master
''')

results = cursor.fetchall()
table = PrettyTable()
table.field_names = [description[0] for description in cursor.description]

for row in results:
    table.add_row(row)
display(table)

tbl_name,sql
mngmnt,"CREATE TABLE mngmnt ( org_management_rk BIGINT, management_nm TEXT)"
product,"CREATE TABLE product ( hit_rk BIGINT, hid BIGINT, using_flg FLOAT)"
queue,"CREATE TABLE queue ( queue_id BIGINT, queue_desc TEXT)"
result,"CREATE TABLE result ( hit_status_result_id BIGINT, hit_status_result_desc TEXT)"
horoscope,"CREATE TABLE horoscope ( agent_login TEXT, horoscope TEXT)"
group,"CREATE TABLE ""group"" ( org_group_rk BIGINT, org_management_rk BIGINT, group_nm TEXT)"
emp_x_org_gr,"CREATE TABLE emp_x_org_gr ( employee_rk BIGINT, org_group_rk BIGINT)"
task,"CREATE TABLE task ( task_rk BIGINT, task_stage_id FLOAT, source_system_cd TEXT, create_dttm TEXT, finish_dttm TEXT)"
action,"CREATE TABLE action ( hit_rk BIGINT, hit_status_result_id BIGINT)"
call,"CREATE TABLE call ( wo_hit_rk BIGINT, wo_task_rk BIGINT, wo_queue_id BIGINT, agent_login TEXT, wo_employee_rk BIGINT, finish_dttm TEXT, duratoin_sec TEXT)"


# Объединение таблиц
Необходимо с помощью SQL запроса собрать одну общую таблицу из всех данных витрин, которая должна включать в себя все поля этих витрин.

## Решение

Проверка уникальных записей ключей

In [6]:
# при помощи расширения sql
# %%sql
# select
#   count (distinct org_management_rk)
# from
#   mngmnt

# при помощи connection
cursor.execute('''
SELECT
  COUNT(DISTINCT org_management_rk)
FROM
  mngmnt
''')

results = cursor.fetchall()
table = PrettyTable()
table.field_names = [description[0] for description in cursor.description]

for row in results:
    table.add_row(row)
display(table)

COUNT(DISTINCT org_management_rk)
45


call - count (distinct wo_task_rk) - 50000
<br>task - count (distinct task_rk) - 50000

call - count (distinct wo_hit_rk) - 89174
<br>action - count (distinct hit_rk) - 89174
<br>product - count (distinct hit_rk) - 89174

call - count (distinct wo_employee_rk) - 1962
<br>emp_x_org_gr - count (distinct employee_rk) - 1962

call - count (distinct wo_queue_id) - 157
<br>queue - count (distinct queue_id) - 155

call - count (distinct agent_login) - 1962<br>horoscope - count (distinct agent_login) - 1962

action - count (distinct hit_status_result_id) - 7<br>result - count (distinct hit_status_result_id) - 7

emp_x_org_gr - count (distinct org_group_rk) - 52<br>group - count (distinct org_group_rk) - 52

group - count (org_management_rk) - 45<br>mngmnt - count (org_management_rk) - 45

Выводы:
- Использование полного внешнего объединения невозможно
- Будет использование внешнее объединение по левой таблице

Вывод 10-ти строк объединённой таблицы

In [7]:
# при помощи расширения sql
# %%sql
# select
#   *
# from
#   call
#   left join task on call.wo_task_rk = task.task_rk
#   left join action on call.wo_hit_rk = action.hit_rk
#   left join product on call.wo_hit_rk = product.hit_rk
#   left join emp_x_org_gr on call.wo_employee_rk = emp_x_org_gr.employee_rk
#   left join queue on call.wo_queue_id = queue.queue_id
#   left join horoscope on call.agent_login = horoscope.agent_login
#   left join result on action.hit_status_result_id = result.hit_status_result_id
#   left join "group" on emp_x_org_gr.org_group_rk = "group".org_group_rk
#   left join mngmnt on "group".org_management_rk = mngmnt.org_management_rk
# limit
#   10;

# при помощи connection
cursor.execute('''
SELECT 
  * 
FROM
  call
  left join task on call.wo_task_rk = task.task_rk
  left join action on call.wo_hit_rk = action.hit_rk
  left join product on call.wo_hit_rk = product.hit_rk
  left join emp_x_org_gr on call.wo_employee_rk = emp_x_org_gr.employee_rk
  left join queue on call.wo_queue_id = queue.queue_id
  left join horoscope on call.agent_login = horoscope.agent_login
  left join result on action.hit_status_result_id = result.hit_status_result_id
  left join "group" on emp_x_org_gr.org_group_rk = "group".org_group_rk
  left join mngmnt on "group".org_management_rk = mngmnt.org_management_rk
LIMIT
  10
''')

# Сохранение запроса в переменную
results = cursor.fetchall()
table = PrettyTable()
# Создание уникальных названий колонок
table.field_names = [f"{description[0]}_{index}" for index, description in enumerate(cursor.description)]
# Вывод результата
for row in results:
    table.add_row(row)
display(table)

wo_hit_rk_0,wo_task_rk_1,wo_queue_id_2,agent_login_3,wo_employee_rk_4,finish_dttm_5,duratoin_sec_6,task_rk_7,task_stage_id_8,source_system_cd_9,create_dttm_10,finish_dttm_11,hit_rk_12,hit_status_result_id_13,hit_rk_14,hid_15,using_flg_16,employee_rk_17,org_group_rk_18,queue_id_19,queue_desc_20,agent_login_21,horoscope_22,hit_status_result_id_23,hit_status_result_desc_24,org_group_rk_25,org_management_rk_26,group_nm_27,org_management_rk_28,management_nm_29
73292106,93625442,12245,lyangama,66749466,2022-08-01 12:52:25,0,93625442,1.0,GI,2022-08-01 00:46:37,2022-08-01 15:35:39,73292106,5,73292106,4,None,66749466,3244,12245,Очередь 143,lyangama,Козерог,5,Недозвон,3244,1621,Группа 30,1621,Управление КЦ Чизан
93763347,98572816,11696,enkoiion,88931008,2022-08-08 10:46:46,"12,53",98572816,1.0,GI,2022-08-06 11:24:23,2022-08-17 11:23:53,93763347,2,93763347,3,None,88931008,2568,11696,Очередь 4,enkoiion,Козерог,2,"Дозвон, Перезвонить",2568,1728,Группа 20,1728,Управление КЦ Мас
13988704,91292760,14354,kinalapo,61041016,2022-08-16 14:16:03,0,91292760,1.0,GI,2022-07-29 04:46:37,2022-09-22 10:57:30,13988704,5,13988704,4,None,61041016,2389,14354,Очередь 58,kinalapo,Козерог,5,Недозвон,2389,323,Группа 5,323,Управление КЦ Ромашка
28495290,12470192,14303,kinasgsa,98181009,2022-08-22 18:38:13,0,12470192,1.0,GI,2022-08-21 01:55:57,2022-08-30 12:59:05,28495290,5,28495290,4,None,98181009,2097,14303,Очередь 79,kinasgsa,Козерог,5,Недозвон,2097,687,Группа 19,687,Управление КЦ Верс
31412029,161228,9264,ikovnisu,47451124,2022-08-23 10:38:24,"128,47",161228,1.0,GI,2022-08-08 01:01:35,2022-08-23 10:38:33,31412029,1,31412029,2,None,47451124,2105,9264,Очередь 43,ikovnisu,Козерог,1,"Дозвон, Отказ",2105,689,Группа 31,689,Управление КЦ Казан
82299889,85191861,14644,atovksip,77811117,2022-08-04 10:53:26,0,85191861,1.0,GI,2022-07-22 17:38:27,2022-08-12 14:57:49,82299889,5,82299889,4,None,77811117,2136,14644,Очередь 119,atovksip,Козерог,5,Недозвон,2136,696,Группа 48,696,Управление КЦ Неп
89329077,98650151,12245,oyankrsh,88658588,2022-08-06 13:32:42,0,98650151,1.0,GI,2022-08-06 06:30:48,2022-08-09 15:49:22,89329077,5,89329077,4,None,88658588,3240,12245,Очередь 143,oyankrsh,Козерог,5,Недозвон,3240,1621,Группа 36,1621,Управление КЦ Чизан
19715922,9596828,14644,llinirra,59539859,2022-08-18 13:19:21,"71,99",9596828,1.0,GI,2022-08-18 00:47:21,2022-08-18 13:19:26,19715922,1,19715922,4,None,59539859,2138,14644,Очередь 119,llinirra,Козерог,1,"Дозвон, Отказ",2138,1521,Группа 29,1521,Управление КЦ Дирим
42214192,14854023,13044,novabbmu,11371111,2022-08-26 11:47:11,"208,12",14854023,23.0,FW,2022-08-24 04:11:24,2022-08-26 11:47:11,42214192,1,42214192,4,None,11371111,960,13044,Очередь 111,novabbmu,Козерог,1,"Дозвон, Отказ",960,402,Группа 15,402,Управление КЦ Шоколад
51183929,18611189,13045,ievavamu,4241040,2022-08-30 16:45:57,0,18611189,23.0,FW,2022-08-30 03:10:40,2022-08-31 10:05:40,51183929,5,51183929,4,None,4241040,58,13045,Очередь 29,ievavamu,Козерог,5,Недозвон,58,325,Группа 42,325,Управление КЦ Супер


# Расчёт метрик
Необходимо посчитать следующие значения:

* Общее количество звонков со статусом «Дозвон, Успешно»;
* Продажная конверсия, по формуле:

  $$ Конверсия = \frac{N_{Дозвон, Успешно}}{N_{Дозвон, Успешно} + N_{Дозвон, Отказ}}$$
* Средняя длительность одной коммуникации, закрытой статусом «Дозвон, Успешно»

Ответы необходимо вписать в соответствующую таблицу в фаиле "Техническое задание"


## Решение

### Общее количество звонков со статусом «Дозвон, Успешно

In [8]:
# при помощи расширения sql
# %%sql
# select
#   count(*)
# from
#   call
#   left join action on call.wo_hit_rk = action.hit_rk
#   left join result on action.hit_status_result_id = result.hit_status_result_id
# where
#   action.hit_status_result_id = 3

# при помощи connection
cursor.execute('''
SELECT 
  COUNT(*)
FROM
  call
  left join action on call.wo_hit_rk = action.hit_rk
  left join result on action.hit_status_result_id = result.hit_status_result_id
WHERE
  action.hit_status_result_id = 3
''')

# Сохранение запроса в переменную
results = cursor.fetchall()
successfull_calls = PrettyTable()

# Вывод результата
for row in results:
    successfull_calls.add_row(row)
print('Количество звонков со статусом "Дозвон, Успешно":', 
      int(float(successfull_calls.get_string().split("|")[3].strip())))

Количество звонков со статусом "Дозвон, Успешно": 3310


### Расчёт продажной конверсии

Дозвон, Отказ

In [9]:
# при помощи расширения sql

# %%sql unsuccessfull_calls <<
# select
#   count(*)
# from
#   call
#   left join action on call.wo_hit_rk = action.hit_rk
#   left join result on action.hit_status_result_id = result.hit_status_result_id
# where
#   action.hit_status_result_id = 1

# при помощи connection
cursor.execute('''
SELECT 
  COUNT(*)
FROM
  call
  left join action on call.wo_hit_rk = action.hit_rk
  left join result on action.hit_status_result_id = result.hit_status_result_id
WHERE
  action.hit_status_result_id = 1
''')

# Сохранение запроса в переменную
results = cursor.fetchall()
unsuccessfull_calls = PrettyTable()

# Вывод результата
for row in results:
    unsuccessfull_calls.add_row(row)
print('Количество звонков со статусом "Дозвон, Отказ":', 
      int(float(unsuccessfull_calls.get_string().split("|")[3].strip())))

Количество звонков со статусом "Дозвон, Отказ": 18459


In [10]:
# при помощи расширения sql
# sales_conversion = successfull_calls[0][0] / (successfull_calls[0][0] + unsuccessfull_calls[0][0])

# при помощи connection
sales_conversion = (
    float(successfull_calls.get_string().split("|")[3].strip()) / 
    (float(successfull_calls.get_string().split("|")[3].strip()) + 
     float(unsuccessfull_calls.get_string().split("|")[3].strip()))
)
print('Продажная конверсия:', round(sales_conversion, 2))

Продажная конверсия: 0.15


### Средняя длительность одной коммуникации, закрытой статусом «Дозвон, Успешно»

In [11]:
# при помощи расширения sql
# %%sql
# select
#   round(avg(duratoin_sec), 0)
# from
#   call
#   left join action on call.wo_hit_rk = action.hit_rk
#   left join result on action.hit_status_result_id = result.hit_status_result_id
# where
#   action.hit_status_result_id = 3;

# при помощи connection
cursor.execute('''
SELECT 
  ROUND(AVG(duratoin_sec), 0)
FROM
  call
  left join action on call.wo_hit_rk = action.hit_rk
  left join result on action.hit_status_result_id = result.hit_status_result_id
WHERE
  action.hit_status_result_id = 3
''')

# Сохранение запроса в переменную
results = cursor.fetchall()
table = PrettyTable()

# Вывод результата
for row in results:
    table.add_row(row)
print('Средняя длительность одной коммуникации, закрытой статусом «Дозвон, Успешно»:', 
      float(table.get_string().split("|")[3].strip()))

Средняя длительность одной коммуникации, закрытой статусом «Дозвон, Успешно»: 110.0


# Закрытие подключения к базе данных

In [12]:
connection.close()

Создание файла requirements.txt

In [13]:
# !pipreqsnb operator_performance_analytics.ipynb

# Заключение
- Исследование провдено в БД SQLite
- Сформированы запросы
- Расчитаны необходимые метрики
- Построен дашборд в Looker Studio
- Изучена продажная конверсия